In [0]:
!pip install kafka-python

In [0]:
dbutils.library.restartPython()

In [0]:
# Pure Python implementation, spark is not used
# bin/kafka-topics.sh --create --bootstrap-server sih-60.cent.uw.edu.pl:49092 --partitions 1 --topic bigdata-test-26
# bin/kafka-console-producer.sh --broker-list sih-60.cent.uw.edu.pl:49092 --topic bigdata-test-26
from kafka import KafkaConsumer
import sys
import time

consumer = KafkaConsumer('bigdata-test-26', bootstrap_servers=['sih-60.cent.uw.edu.pl:49092'], group_id='my_consumer')

for message in consumer:
  print(message.value.decode('utf-8'))
  time.sleep(1)

In [0]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "sih-60.cent.uw.edu.pl:49092") \
  .option("subscribe", "bigdata-test-26") \
  .load() \
  .selectExpr("CAST(value AS STRING) as message")
  
#.option("value.deserializer", ) \
#df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
display(df, checkpointLocation = "/Workspace/Users/pawel.rubach@sgh.waw.pl/")

In [0]:
from pyspark.sql.types import *

movies_schema = StructType([
  StructField('movieId', IntegerType()),
  StructField('title', StringType()),
  StructField('genres', StringType())
])

file_location = "/databricks-datasets/cs110x/ml-1m/data-001/movies.dat"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "false"
delimiter = "::"

# The applied options are for CSV files. For other file types, these will be ignored.
df_movies = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .schema(movies_schema) \
  .load(file_location)

display(df_movies)

In [0]:
# ./kafka-console-consumer.sh --bootstrap-server sih-60.cent.uw.edu.pl:49092 --topic bigdata-test-26
#spark.conf.set("spark.sql.streaming.stopActiveRunOnRestart", True) 
ds = df_movies \
  .selectExpr("CAST(movieId AS STRING)", "CAST(title AS STRING) as value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "sih-60.cent.uw.edu.pl:49092") \
  .option("topic", "bigdata-test-26") \
  .save()